<a href="https://colab.research.google.com/github/bacoco/LLM_train/blob/main/Claude_3_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install llama-index
%pip install transformers
%pip install llama-index-readers-web
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-anthropic

## Setup

### Data

In [ ]:
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

documents = BeautifulSoupWebReader().load_data([url])

### LLM

In [ ]:
import os

os.environ['ANTHROPIC_API_KEY'] = "sk-"

In [ ]:
from llama_index.llms.anthropic import Anthropic

# llm = Anthropic(model="claude-3-sonnet-20240229", temperature=0.0)
llm = Anthropic(model="claude-3-opus-20240229", temperature=0.0)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

### Index Setup

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

### Helpful Imports / Logging

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** Based on the information provided, OpenAI and Meta are taking somewhat different approaches with their AI tools:

OpenAI is positioning ChatGPT more as a productivity tool - a utility for getting things done. The recent addition of voice interaction and image analysis capabilities make ChatGPT more powerful and useful in a wider range of scenarios. The voice feature in particular hints at ChatGPT's potential to serve as an AI companion.

In contrast, Meta is focusing more on the entertainment potential of AI. They have unveiled personality-driven AI chatbots that use the voices of celebrities like Snoop Dogg and Kendall Jenner. These seem designed more for novelty and fan engagement rather than productivity. 

So while OpenAI is building general-purpose AI assistants, Meta is creating specific AI characters meant to entertain. However, both companies are exploring how voice interaction and virtual personas could change how people interact with AI systems in their products and platforms going forward. The article suggests this could lead to more "synthetic" social networks.

### Refine

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

**`Final Response:`** Based on the context, OpenAI and Meta differ in their approach to AI tools in a few key ways:

1. Purpose - OpenAI tends to present its AI products like ChatGPT as productivity tools for getting things done. In contrast, Meta is more focused on using AI for entertainment purposes in its social apps.

2. Voice AI - OpenAI has recently added voice interaction capabilities to ChatGPT, allowing users to speak with the AI model using one of five native voices. Meta has created personality-driven AI chatbots that use celebrity voices to interact with users in its messaging apps. 

3. Integration - Meta plans to deeply integrate its AI characters across all its major products, including giving them Facebook pages, Instagram accounts, and the ability to message users. OpenAI's tools are more standalone utilities rather than being embedded into a larger social network.

4. Synthetic Personas - Meta is experimenting with giving its AI chatbots synthetic personalities, whereas OpenAI's tools aim to be more general purpose assistants without a defined persona, although the addition of voice is starting to give ChatGPT a hint of personality.

So in summary, while both companies are exploring voice-based AI, OpenAI is focused on enhancing its productivity-oriented tools with voice interaction, while Meta is doubling down on personality-driven AI entertainment that is tightly woven into its social platforms.

### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

## Router Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=False,
)

# this doesn't work
# response = query_engine.query("What was mentioned about Meta?")

# this works
response = query_engine.query("What was mentioned about Meta? Use a tool.")

display_response(response)

**`Final Response:`** According to the article, Meta is doing the following related to AI:

- Meta is building large language models (LLMs), like OpenAI, but is focused more on using them for entertainment rather than just productivity.

- Meta unveiled 28 personality-driven AI chatbots that will be used in its messaging apps. These bots use the voices of celebrities like Charli D'Amelio, Snoop Dogg, Tom Brady, and Paris Hilton. 

- The celebrity AI characters come with brief descriptions, like MrBeast's bot Zach being described as "the big brother who will roast you — because he cares."

- Meta plans to place these AI characters on every major surface of its products - they will have Facebook pages, Instagram accounts, and users can message them in the same inbox as friends and family. Soon they may also be making Reels videos.

- In addition to the chatbots, Meta also announced AI-generated stickers that will be coming to its messaging apps.

So in summary, Meta is investing heavily in using generative AI and voice technology to create a range of entertaining AI characters and features that will be integrated throughout its social platforms. This represents a shift towards feeds becoming a "partially synthetic social network."

In [ ]:
len(response.source_nodes)

2

### Multi Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True,
)

response = query_engine.query("What was mentioned about Meta? Summarize with any other companies mentioned in the entire document.")

display_response(response)

**`Final Response:`** According to the article, Meta is building large language models (LLMs) and has found uses for generative AI and voices in its products. The company unveiled 28 personality-driven chatbots with celebrity voices like Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton to be used in its messaging apps. Meta plans to place these AI characters on every major surface of its products, including Facebook pages, Instagram accounts, and messaging inboxes. The article suggests this could lead to a partially synthetic social network.

The article also mentions OpenAI, which recently announced updates to ChatGPT that allow voice interaction and image uploading for questions. The author believes adding a voice to ChatGPT begins to give it a hint of personality, and could lead to AI companions that are smarter, more patient, empathetic and available than real people. However, OpenAI tends to present its products more as productivity tools compared to Meta's entertainment focus.

## SubQuestion Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

Generated 5 sub questions.
[vector_search] Q: What was mentioned about Meta in the document?
[summary] Q: Summarize the key points about how Meta is discussed in the document.
[vector_search] Q: What was mentioned about OpenAI in the document?
[summary] Q: Summarize the key points about how OpenAI is discussed in the document.
[summary] Q: Compare and contrast the key differences in how Meta and OpenAI are discussed based on the summaries.
[vector_search] A: According to the document, Meta unveiled the following related to generative AI:

- In addition to an all-purpose AI assistant, Meta revealed 28 personality-driven chatbots to be used in its messaging apps. 

- Celebrities like Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lent their voices to these chatbots. Each chatbot character comes with a brief description.

- Meta plans to place its AI characters on every major surface of its products, including giving them Facebook pages, Ins

**`Final Response:`** According to the article, Meta is taking a different approach to generative AI compared to OpenAI:

1. Meta unveiled 28 personality-driven chatbots voiced by celebrities like Charli D'Amelio, Snoop Dogg, and Paris Hilton to be used in its messaging apps. This entertainment focus contrasts with OpenAI positioning products like ChatGPT as productivity tools.

2. Meta plans to deeply integrate its AI characters across its products, giving them Facebook pages, Instagram accounts, and allowing users to message them alongside friends and family. This could make social media feeds partially synthetic. The article doesn't mention similar social network integration plans for OpenAI.

3. While OpenAI's speaking ChatGPT is seen as an early step towards empathetic AI companions, Meta's celebrity-voiced characters seem more focused on novelty and entertainment value currently. 

4. Meta is also integrating AI-generated imagery like stickers in its messaging apps. OpenAI recently added the ability to upload images to ChatGPT to ask questions about them.

In summary, Meta appears to be leaning into the entertainment potential of generative AI with celebrity voices and prominent placement in its social products, while OpenAI is focused more on enhancing productivity with features that make ChatGPT more useful and relatable. However, both companies are investing heavily in large language models and voice interaction.

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1991k      0 --:--:-- --:--:-- --:--:-- 2002k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True
)

In [ ]:
response = query_engine.query("What are some albums?")

display_response(response)

**`Final Response:`** Here are some album titles from the query results:

1. ...And Justice For All
2. 20th Century Masters - The Millennium Collection: The Best of Scorpions  
3. A Copland Celebration, Vol. I
4. A Matter of Life and Death
5. A Real Dead One

The query selects the Title column from an albums table, orders the results alphabetically by title, and limits the output to 5 rows.

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

**`Final Response:`** Here are 5 artists from the database:

1. AC/DC
2. Accept 
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

**`Final Response:`** Here are 3 tracks from the artist AC/DC:

1. "For Those About To Rock (We Salute You)" from the album For Those About To Rock We Salute You
2. "Put The Finger On You" from the album For Those About To Rock We Salute You  
3. "Let's Get It Up" from the album For Those About To Rock We Salute You

In [ ]:
print(response.metadata['sql_query'])

SELECT tracks.Name, albums.Title 
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'AC/DC'
LIMIT 3;


## Programs

We use our `LLMTextCompletionProgram`, which uses standard prompting to coerce the model into outputting JSON.

Claude 3 of course has function calling support, testing that is TBD.

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.core.program import LLMTextCompletionProgram
from llama_index.program.openai import OpenAIPydanticProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_cls=Album,
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

In [ ]:
print(output)
print(type(output))

name='Overlook Hotel' artist='The Torrance Trio' songs=[Song(title='All Work and No Play', length_seconds=243), Song(title='Redrum Lullaby', length_seconds=186), Song(title="Here's Johnny!", length_seconds=157), Song(title='Midnight Maze', length_seconds=301), Song(title='Elevator of Blood', length_seconds=222), Song(title='Twins in the Hallway', length_seconds=179), Song(title='Typewriter Serenade', length_seconds=289), Song(title='Room 237 Blues', length_seconds=213), Song(title='Axe Swing Waltz', length_seconds=197), Song(title='Snowy Hedge Maze', length_seconds=312)]
<class '__main__.Album'>


## Data Agent

We use our `ReActAgent`.

In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

In [ ]:
response = agent.chat("Hello!")
print(response)

Thought: I don't need any tools to respond to a simple greeting.
Answer: Hello! How can I assist you today?
Hello! How can I assist you today?


In [ ]:
# vector_tool.call(input="What was mentioned about Meta? How does it differ from how OpenAI is talked about?")

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Thought: To answer this question, I will need to search the provided documents for mentions of Meta and OpenAI and summarize how they are discussed differently.
Action: vector_search
Action Input: {'input': 'Meta'}
Observation: Meta recently unveiled 28 personality-driven AI chatbots for its messaging apps, featuring the voices of celebrities like Charli D'Amelio, Snoop Dogg, and Paris Hilton. Each chatbot has a unique character - for example, MrBeast's bot is described as "the big brother who will roast you — because he cares." 

Meta plans to integrate these AI characters across its products, giving them Facebook pages, Instagram accounts, and the ability to message users directly. This represents a new direction for social networking, where synthetic AI-powered personas will increasingly populate feeds alongside content from real human connections.

It's still early days for this technology. The current celebrity chatbots seem more like a novelty, and the celebrities themselves like